In [1]:
import pandas
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import norm
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.base.model import GenericLikelihoodModel

In [3]:
#load data into memory
data3 = np.genfromtxt('data3.dat', delimiter='  ')
data3 = data3.transpose()

#test to see if other data works
drawsml = np.genfromtxt('drawsml.dat', delimiter='  ')
#drawsgmm = np.genfromtxt('drawsgmm.dat', delimiter='  ')

In [4]:
def sim_ll(y,x1,x2,t1,t2,sigma,v=False):
    """Simulate the log likelihood function by integrating out"""
    #hard coded draws
    eps1 = np.genfromtxt('drawsml.dat', delimiter='  ').transpose()
    sim_like = 0 
    #set up vectors for adding
    zeros = np.full(len(x1),.0001)
    const = np.full(len(x1), 100)   
    
    #run simulations, to simulate likelihoods
    for s in range(len(eps1)): 
        #calculate f inverse
        mc1 = np.exp(t1 + t2*x1 + sigma*eps1[s])
        safe_log1 = np.maximum( 3*y - const - mc1 ,zeros)
        f_inv =  (np.log(safe_log1) - t1 - t2*x2)
        safe_f_inv = np.maximum( f_inv ,zeros)
        #calculate f prime
        f_prime = (3.0/sigma)*1/(3.0*y - const - mc1)
        sim_like = sim_like + norm(0,sigma).pdf(f_inv)*f_prime
    
    #finalized simulated likelihoods
    safe_sim = np.maximum(sim_like/len(eps1),zeros)    
    return np.log(safe_sim).sum()


class part_b(GenericLikelihoodModel):
    """class for evaluating question 3 part b"""
    
    def nloglikeobs(self, params):
        t1, t2, sigma = params
        y = self.endog
        x1,x2 = self.exog.transpose()
        return -sim_ll(y, x1,x2,t1,t2,sigma)

    
    def fit(self, start_params=None, maxiter=100, maxfun=5000, **kwds):
        if start_params == None:
            start_params = np.array([1,1,1])
        return super(part_b, self).fit(start_params=start_params,
                                       maxiter=maxiter, maxfun=maxfun, **kwds)

    
model_b = part_b(data3[0],data3[1:].transpose()).fit(start_params = [1,1,1])
print(model_b.summary(xname=['theta_1','theta_2','sigma']))

Optimization terminated successfully.
         Current function value: -3.174360
         Iterations: 92
         Function evaluations: 166
                                part_b Results                                
Dep. Variable:                      y   Log-Likelihood:                 158.72
Model:                         part_b   AIC:                            -313.4
Method:            Maximum Likelihood   BIC:                            -309.6
Date:                Sat, 24 Jun 2023                                         
Time:                        00:51:03                                         
No. Observations:                  50                                         
Df Residuals:                      48                                         
Df Model:                           1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------

c:\Users\ccerl\Anaconda3\envs\empiricalio\Lib\site-packages\statsmodels\base\model.py:2742: UserWarning: df_model + k_constant + k_extra differs from k_params
  warnings.warn("df_model + k_constant + k_extra "
c:\Users\ccerl\Anaconda3\envs\empiricalio\Lib\site-packages\statsmodels\base\model.py:2746: UserWarning: df_resid differs from nobs - k_params
  warnings.warn("df_resid differs from nobs - k_params")


In [6]:
def sim_g(t1,t2,sigma,y,x1,x2,eps1,eps2):
    """simulate moment conditions, given predetermined
    simulations"""
    g = 0
    for s in range(len(eps1)):
        y_hat = 1/3.0*(100 + np.exp(t1 + t2*x1 + sigma*eps1[s])
                       + np.exp(t1 + t2*x2 + sigma*eps2[s]))
        g = g + (y - y_hat)
        
    return g/ (len(eps1)*len(eps2))


def sim_g_sq(t1,t2,sigma,y,x1,x2,eps1,eps2):
    """simulate square moment conditions, given predetermined
    simulations"""
    g = 0
    for s in range(len(eps1)):
        y_hat = 1/3.0*(100 + np.exp(t1 + t2*x1 + sigma*eps1[s])
                       + np.exp(t1 + t2*x2 + sigma*eps2[s]))
        g = g + (y**2-y_hat**2)
    return g/(len(eps1)*len(eps2))


class part_d(GMM):
    """class for evaluating question 3 part d"""

    def __init__(self, *args, **kwds):
        """set appropriate counts for moment conditions and parameters"""
        kwds.setdefault('k_moms', 4)
        kwds.setdefault('k_params',3)
        super(part_d, self).__init__(*args, **kwds)
    
    
    def fit(self, start_params=None, maxiter=10000, **kwds):
        if start_params == None:
            start_params = np.array([.5, .5, .5])
        return super(part_d, self).fit(start_params=start_params,
                                       maxiter=maxiter,  **kwds)

        
    def momcond(self, params):
        t1,t2, sigma = params   
        y = self.endog
        x1,x2 = self.exog.transpose() 
        
        #initialize draws
        #eps = np.genfromtxt('drawsgmm.dat', delimiter='  ').transpose()
        eps = np.random.normal(0,1,(50,40)).transpose()
        eps1 = eps[:20,:]
        eps2 = eps[20:,:]
        
        #calc moment conditions
        g = sim_g(t1,t2,sigma,y,x1,x2,eps1,eps2)
        g_sq = sim_g_sq(t1,t2,sigma,y,x1,x2,eps1,eps2)
        return np.column_stack((g,g*x1,g*x2,g_sq))


model_d = part_d(data3[0],data3[1:].transpose(), None)


#print first iteration (i.e. 1 stage)
result_d_stage1 = model_d.fit(maxiter=1, optim_method='nm', wargs=dict(centered=False))
print(result_d_stage1.summary(xname=['theta_1','theta_2','sigma']))
print('\n')


#print second iteration (i.e. 2 stage)
result_d_stage2 = model_d.fit(maxiter=2, optim_method='nm', wargs=dict(centered=False))
print(result_d_stage2.summary(xname=['theta_1','theta_2','sigma']))

c:\Users\ccerl\Anaconda3\envs\empiricalio\Lib\site-packages\statsmodels\sandbox\regression\gmm.py:768: RuntimeWarning: Maximum number of function evaluations has been exceeded.
  return optimizer(self.gmmobjective, start, args=(weights,),


                                part_d Results                                
Dep. Variable:                      y   Hansen J:                        191.5
Model:                         part_d   Prob (Hansen J):              1.46e-43
Method:                           GMM                                         
Date:                Sat, 24 Jun 2023                                         
Time:                        00:54:54                                         
No. Observations:                  50                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
theta_1        0.5330      0.001    770.843      0.000       0.532       0.534
theta_2        0.4834      0.001    731.062      0.000       0.482       0.485
sigma          0.4864      0.001    709.733      0.000       0.485       0.488


Optimization terminated successfully.
         Cur

c:\Users\ccerl\Anaconda3\envs\empiricalio\Lib\site-packages\statsmodels\sandbox\regression\gmm.py:768: RuntimeWarning: Maximum number of function evaluations has been exceeded.
  return optimizer(self.gmmobjective, start, args=(weights,),


In [7]:
result_d_stage2 = model_d.fit(maxiter=10000, optim_method='nm', wargs=dict(centered=False))
print(result_d_stage2.summary(xname=['theta_1','theta_2','sigma']))

c:\Users\ccerl\Anaconda3\envs\empiricalio\Lib\site-packages\statsmodels\sandbox\regression\gmm.py:768: RuntimeWarning: Maximum number of function evaluations has been exceeded.
  return optimizer(self.gmmobjective, start, args=(weights,),


                                part_d Results                                
Dep. Variable:                      y   Hansen J:                        1.488
Model:                         part_d   Prob (Hansen J):                 0.223
Method:                           GMM                                         
Date:                Sat, 24 Jun 2023                                         
Time:                        00:55:44                                         
No. Observations:                  50                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
theta_1        1.0019      0.000   2925.882      0.000       1.001       1.003
theta_2        1.1064      0.001   1461.474      0.000       1.105       1.108
sigma          0.0989      0.001    162.636      0.000       0.098       0.100
